In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.sparse as sp
import scipy.sparse.linalg as la
import matplotlib.ticker as ticker

In [2]:
def sourcefunc_2D(x,y):
    f = (x**2 + y**2)*np.sin(x*y)
    return f

def boundary_2D(x,y):
    b = np.sin(x*y)
    return b

def sourcefunc_3D(x,y,z):
    f = ((y**2)*(x**2) + (y**2)*(z**2) + (z**2)*(x**2))*np.sin(x*y*z)
    return f

def boundary_3D(x,y,z):
    b = np.sin(x*y*z)
    return b

In [10]:
def create_Afuex_2D(sourcefunc_2D, boundary_2D, p):

    # Here I create the T_h and I_h matrices. These have precisely the same form as in the lecture notes. Some manual
    # stuff is done since we are working without elimination of boundary conditions
    h = 1/(2**p)
    N = 1/h
    N = int(N)
    Th = sp.diags([-1, 2, -1], [-1, 0, 1], shape=(N-1, N-1)).toarray()
    T_h = np.pad(Th,1)
    T_h[0,0] = h**2
    T_h[N,N] = h**2
    Ih = sp.diags([1],[0], shape=(N-1, N-1)).toarray()
    I_h = np.pad(Ih,1)
    # The final A_h matrix is construced here. Because of the h^2 * I_{N+1} identity matrix in the very top left 
    # corner and bottom right corner I have to change four values manually from zero to 1
    A_2D = (1/(h**2))*np.kron(T_h, I_h) + (1/(h**2))*np.kron(I_h, T_h)
    A_2D[0,0] = 1
    A_2D[N,N] = 1
    A_2D[(N+1)**2-N-1,(N+1)**2-N-1] = 1
    A_2D[(N+1)**2-1,(N+1)**2-1] = 1
    
    
    # A meshgrid is created here on which I will evalute the source function. This vector is the right size for
    # the final result, but it includes every boundary value also, as evaluated through f. This is obviously wrong
    # as these boundary values should be evaluated through b, so that has to be adjusted. I therefore immediately 
    # introduce b1 and b_end as vectors which are the boundary values on the bottom and top of the grid, respectively.
    # f is also reshaped here to be a vector, not an array.
    x,y = np.mgrid[0: 1: complex(0, N+1), 0: 1: complex(0, N+1)]
    x = x.transpose()
    y = y.transpose()

    f_2D = sourcefunc_2D(x,y)
    f_2D = np.reshape(f_2D, (N+1)*(N+1))

    x_axis = np.linspace(0, 1, num = N+1)
    b1 = boundary_2D(x_axis, 0)
    b_end = boundary_2D(x_axis, 1)
    
    # In this section I overwrite the parts of the f vector that represent boundary terms and next-to-boundary terms.
    # In the first loop I overwrite the first and last parts of f with b1 and b_end, so that the bottom and top of the 
    # 'grid' are boundary values. In the second loop I overwrite values representing the left and right side of the
    # 'grid'. Of course the bottom and left boundaries are just filled with zeros, as sin(xy) is zero when either x
    # or y is zero. In the third loop I overwrite the entries which represent positions next to the right boundary. In
    # the last loop I overwrite the entries which represent positions right below the top boundary. 


    for i in range(0, N+1):
        f_2D[i] = b1[i]
        f_2D[(N+1)*N + i] = b_end[i]

    for i in range(1,N):
        f_2D[i*(N+1)] = 0
        f_2D[i*(N+1)+ N] = boundary_2D(1, i*h)
    
    for i in range(0,N-1):    
        f_2D[2*N+i*(N+1)] = f_2D[2*N+i*(N+1)] + boundary_2D(1, (i+1)*h)/(h**2)
    
    for i in range(0,N-1):     
        f_2D[(N+1)**2-1-2*N+i] = f_2D[(N+1)**2-1-2*N+i] + b_end[i+1]/(h**2)
        
    u_ex_pre_2D = boundary_2D(x,y)
    u_ex_2D = np.reshape(u_ex_pre_2D, (1, (N+1)*(N+1)))
    
    A_2D = sp.csr_matrix(A_2D)

    return A_2D , f_2D , u_ex_2D, N

In [8]:
def create_Afuex_3D(A_2D, sourcefunc_3D, boundary_3D, p):
    
    h = 1/(2**p)
    N = 1/h
    N = int(N)
    A = A_2D.torarray()

    for i in range((N+1)**2):
        if A[i,i] == 1:
            A[i,i] = A[i,i]
        else:
            A[i,i] = 1.5*A[i,i]
    
    
    I_b1 = sp.eye(N-1).toarray()      
    I_b2 = np.pad(I_b1,1)     # Now we have a padded identity of size (N+1) x (N+1)
    I_b3 = sp.csr_matrix(I_b2)

    I_c1 = sp.diags([-1, -1], [-1, 1], shape=(N-1, N-1)).toarray()
    I_c2 = np.pad(I_c1, 1)
    I_c3 = sp.csr_matrix(I_c2)

    I_d = np.zeros((N+1,N+1))
    I_d[0,0] = 1
    I_d[N,N] = 1
    I_d2 = sp.csr_matrix(I_d)

    off_diag = sp.kron(I_b3, I_b3)

    A_3D = sp.kron(I_b3, A) + sp.kron(I_c3, (N**2)*off_diag) + sp.kron(I_d, sp.eye((N+1)**2))
    
    for i in range((N+1)**2):
        if A_3D[i,i] == 0:
            A_3D[i,i] = 1
        else:
            A[i,i] = A_3D[i,i]
            
            
            
            
    z,y,x = np.mgrid[0: 1: complex(0, N+1), 0: 1: complex(0, N+1), 0: 1: complex(0, N+1)]

    f_3D = sourcefunc_3D(x,y,z)
    f_3D = np.reshape(f_3D, (N+1)**3)

    b_3D = boundary_3D(x,y,z)
    b_3D = np.reshape(b_3D, (N+1)**3)
    
    
    for i in range(0, (N+1)**2): # Overwriting the first and last layer of the 3D cubic grid
        f_3D[i] = b_3D[i]
        f_3D[((N+1)**2)*N + i] = b_3D[((N+1)**2)*N + i]
    

    for i in range(0, (N+1)**2): # right before the last layer
        f_3D[(N-1)*(N+1)**2 + i] = f_3D[(N-1)*(N+1)**2 + i] + b_3D[N*(N+1)**2 + i]*(N**2)
        
    
    for j in range(1, N):   # for each layer j 
    
        for i in range(0,N-1): # next to boundary points on right side  
            f_3D[2*N+i*(N+1) + j*(N+1)**2] = f_3D[2*N+i*(N+1) + j*(N+1)**2] + b_3D[2*N+i*(N+1) + j*(N+1)**2 + 1]*(N**2)
    
        for i in range(0,N-1): # right below boundary points on top    
            f_3D[(N+1)**2-1-2*N+i + j*(N+1)**2] = f_3D[(N+1)**2-1-2*N+i + j*(N+1)**2] + b_3D[(N+1)**2-1-2*N+i + j*(N+1)**2 + N + 1]*(N**2)
        
        for i in range(0, N+1): # actual boundary points on top and bottom
            f_3D[i + j*(N+1)**2] = b_3D[i + j*(N+1)**2]
            f_3D[(N+1)*N + i + j*(N+1)**2] = b_3D[(N+1)*N + i + j*(N+1)**2]

        for i in range(1,N): # actual boundary points on sides
            f_3D[i*(N+1) + j*(N+1)**2] = 0
            f_3D[i*(N+1) + j*(N+1)**2 + N] = b_3D[i*(N+1)+ j*(N+1)**2 + N]
            
            
        u_ex_3D = b_3D
    
    return A_3D, f_3D, b_3D, u_ex_3D

In [37]:
#Method of LU decomposition. Returns matrix with L in lower triangular part and U in upper.

def LUfunc(M): 

#   To make sure we have float division
#   A = sparse.csr_matrix(Afloat) #make sparse

    N = A.shape[0]    #length of square matrix
    for k in range(N-1):
        if A[k,k] == 0:
            print("Breakdown due to: Zero pivot")
            break
        for i in range(k+1,N):
            A[i,k] = A[i,k]/A[k,k]
            for j in range(k+1,N):
                A[i,j] = A[i,j] - A[i,k]*A[k,j]
    return A
    
    
def LUfunc2(A):  #Returns LUfunc matrix result as L and U matrices
    L = sp.tril(A,-1)+sp.identity(A.shape[0])
    U = sp.triu(A)
    return L , U

#Forward     Works for a LU decomposed matrix from LUfunc      L is NxN matrix, f is N vector
def Forsub(A,f):
    N = A.shape[0]
    y = sp.csr_matrix(np.zeros(N))
    y[0] = f[0] #first element is always just f[0] due to lower tridiagonal
    L = A
    for i in range(N):
        L[i,i] = 1   #Make sure diagonal is 1
    for i in range(1,N):
        y[i]=f[i]-L[i,:i].dot(y[:i])
    return sp.csr_matrix(y)
    
    #backward    Use y from Forsub
def Backsub(A,y):
    N = A.shape[0]
    U = A
    u = sp.csr_matrix(np.zeros(N))
    u[-1] = y[-1]/U[-1,-1]   #manually add first element to u
    for i in reversed(range(N-1)):
        u[i] = ( y[i]-U[i,i:N].dot(u[i:N]) )/U[i,i]
    return u
    
def DirSolver(A,f): #Combines the LU decompostion and forwards and backwards substitutions 
    LU = LUfunc(A)
    y = Forsub(LU,f)
    u = Backsub(LU,y)
    return u

In [38]:
import time

error_vec_2D = []
error_vec_3D = []
for i in range(2,8):
    # Comment all this out when doing 3D
    A_2D , f_2D, u_ex_2D, N = create_Afuex_2D(sourcefunc_2D, boundary_2D, i)    
    t = time.time()
    #u_2D = la.spsolve(A_2D_csr, f_2D)
    u_2D = DirSolver(A_2D_csr.toarray(),f_2D)
    print("For p:",i,"time is :",time.time()-t)
    error_2D = np.amax(np.absolute(u_2D-u_ex_2D))
    print("p =", i, ": error_2D =", error_2D)
    error_vec_2D.append(error_2D)


    # Comment all this out when doing 2D
    #A_2D , f_2D, u_ex_2D, N = create_Afuex_2D(sourcefunc_2D, boundary_2D, i)    
    #A_3D, f_3D, b_3D , u_ex_3D = create_Afuex_3D(A_2D, sourcefunc_3D, boundary_3D, i)
    #A_3D_csr = sp.csr_matrix(A_3D)
    #u_3D = la.spsolve(A_3D_csr, f_3D)
    #error_3D = np.amax(np.absolute(u_3D-u_ex_3D))
    #print("p =", i, ": error_3D =", error_3D)
    #error_vec_3D.append(error_3D)

IndexError: row index (1) out of range

In [28]:
A = sp.csr_matrix( [[1,0,1],[0,0,0],[0,2,0]] ) 
v = sp.csr_matrix( [[1],[0],[1]] )


print(A[1,1])
print(A[0,:].dot(v))

0
  (0, 0)	2
